In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from mpmath import besseljzero
from scipy.special import gamma
from scipy.special import jv, iv
from scipy.optimize import minimize
from scipy.interpolate import interp1d

from scipy.stats import pearsonr
from sklearn.metrics import r2_score

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def simulate_HSDM_2D(a, mu, ndt, sigma=1, dt=0.001):
    x = np.zeros(mu.shape)
    
    rt = 0
    
    while np.linalg.norm(x, 2) < a(rt):
        x += mu*dt + sigma*np.sqrt(dt)*np.random.normal(0, 1, mu.shape)
        rt += dt
    
    theta = np.arctan2(x[1], x[0])   
    
    return ndt+rt, theta

In [3]:
def series_bessel_fpt(t, a=1, sigma=1, nu=0, n=100):
    zeros = np.asarray([float(besseljzero(nu, i+1)) for i in range(n)])
    fpt = np.zeros(t.shape)
    
    for i in range(t.shape[0]):
        series = np.sum((zeros**(nu+1)/jv(nu+1, zeros)) * np.exp(-(zeros**2 * sigma**2)/(2*a**2)*t[i]))
        fpt[i] = sigma**2/(2**nu * a**2 * gamma(nu + 1)) * series
        
    return interp1d(t, fpt)

In [4]:
def HSDM_2D_likelihood(prms, RT, Theta, N_series):
    a =  prms[0]
    ndt = prms[1]
    mu = np.array([prms[2], prms[3]])
    
    tt = np.arange(0.001, max(RT)+0.02, 0.02)
    fpt = series_bessel_fpt(tt, a, sigma=1, nu=(mu.shape[0]-2)/2, n=N_series)
    
    log_lik = 0
    for i in range(len(RT)):
        rt, theta = RT[i], Theta[i]
        if rt - ndt > 0.001:
            mu_dot_x0 = mu[0]*np.cos(theta)
            mu_dot_x1 = mu[1]*np.sin(theta)
            term1 = prms[0] * (mu_dot_x0 + mu_dot_x1)
            term2 = 0.5 * np.linalg.norm(mu, 2)**2 * (rt - ndt)
            
            density = np.exp(term1 - term2) * fpt(rt - ndt)
            
            if 0.1**14 < density:
                log_lik += -np.log(density)
            else:
                log_lik += -np.log(0.1**14)
        else:
            log_lik += -np.log(0.1**14)
        
    return log_lik

In [5]:
recovery_df = {'threshold_true': [],
               'threshold_estimate': [],
               'ndt_true': [],
               'ndt_estimate': [],
               'mu1_true': [],
               'mu1_estimate': [],
               'mu2_true': [],
               'mu2_estimate': []}

min_threshold = 0.5
max_threshold = 5

min_ndt = 0.1
max_ndt = 1

min_mu = -3.5
max_mu = 3.5

N_series = 150

In [6]:
for n in tqdm(range(150)):
    threshold = np.random.uniform(min_threshold, max_threshold)
    a = lambda t: threshold
    ndt = np.random.uniform(min_ndt, max_ndt)
    mu = np.array([np.random.uniform(min_mu, max_mu), 
                   np.random.uniform(min_mu, max_mu)])
    
    
    recovery_df['threshold_true'].append(threshold)
    recovery_df['ndt_true'].append(ndt)
    recovery_df['mu1_true'].append(mu[0])
    recovery_df['mu2_true'].append(mu[1])
    
    RT = []
    Theta = []
    
    for i in range(250):
        rt, theta = simulate_HSDM_2D(a, mu, ndt)
        RT.append(rt)
        Theta.append(theta)
    
    min_ans = minimize(HSDM_2D_likelihood,
                       args=(RT, Theta, N_series), 
                       x0=np.array([np.random.uniform(min_threshold, max_threshold),
                                    np.random.uniform(min_ndt, max_ndt), 
                                    np.random.uniform(min_mu, max_mu),
                                    np.random.uniform(min_mu, max_mu)]),
                       method='Nelder-Mead', 
                       bounds=[(min_threshold, max_threshold), (min_ndt, max_ndt),
                               (min_mu, max_mu), (min_mu, max_mu)])
    
    recovery_df['threshold_estimate'].append(min_ans.x[0])
    recovery_df['ndt_estimate'].append(min_ans.x[1])
    recovery_df['mu1_estimate'].append(min_ans.x[2])
    recovery_df['mu2_estimate'].append(min_ans.x[3])
    
recovery_df = pd.DataFrame(recovery_df)

100%|███████████████████████████████████████████████| 150/150 [2:42:55<00:00, 65.17s/it]


In [7]:
recovery_df

,threshold_true,threshold_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate
0,2.298864,2.190848,0.814451,0.886454,-1.697464,-1.681382,0.172031,0.090295
1,2.327819,2.327579,0.662403,0.703405,-1.012439,-1.024405,1.083154,1.146525
2,4.170291,4.349488,0.851975,0.789429,-1.187871,-1.113513,3.115605,3.008174
3,4.275393,4.532711,0.523647,0.483867,1.609273,1.618161,-2.596601,-2.636934
4,3.813763,3.444139,0.452422,0.517892,-3.413691,-3.282146,0.207232,0.259685
...,...,...,...,...,...,...,...,...
145,1.415803,1.507449,0.157941,0.150315,3.061932,3.160371,-2.451375,-2.385592
146,1.842121,1.859358,0.838562,0.834747,1.313091,1.344292,0.720319,0.665421
147,1.381568,1.567941,0.985247,0.982519,-2.857139,-3.155957,2.765088,3.500000
148,3.956877,3.885910,0.339332,0.418053,2.238944,2.224025,0.132757,0.140843


In [8]:
recovery_df.corr()

,threshold_true,threshold_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate
threshold_true,1.000000,0.844428,-0.073957,0.055079,-0.053913,-0.030229,-0.069801,-0.034049
threshold_estimate,0.844428,1.000000,-0.070541,-0.231484,-0.016775,-0.063516,-0.089193,-0.046453
ndt_true,-0.073957,-0.070541,1.000000,0.735249,-0.015202,-0.025903,0.081637,0.105536
ndt_estimate,0.055079,-0.231484,0.735249,1.000000,-0.049763,0.000777,0.123910,0.124051
mu1_true,-0.053913,-0.016775,-0.015202,-0.049763,1.000000,0.948620,-0.070723,-0.061069
mu1_estimate,-0.030229,-0.063516,-0.025903,0.000777,0.948620,1.000000,-0.073883,-0.109700
mu2_true,-0.069801,-0.089193,0.081637,0.123910,-0.070723,-0.073883,1.000000,0.973247
mu2_estimate,-0.034049,-0.046453,0.105536,0.124051,-0.061069,-0.109700,0.973247,1.000000


In [9]:
recovery_df.to_csv('Series_2d_recovery_{}.csv'.format(N_series))